In [44]:
import math
import numpy as np
import random

# RSA

- RSA_setup(p,q):
- RSA_encrypt(m,e):
- RSA_decrypt(M,d):

In [45]:
def _ext_euc_alg(a, b):
    '''
    Performs Extended Euclidean Algorithm on two integers

            Parameters:
                    a : An integer
                    b : Another integer

            Returns:
                    (gcd, x, y) : tuple of GCD of (a,b) and (x,y) orthogonal to (a,b)
    '''
    if a == 0:
        return b, 0, 1
             
    gcd,x1,y1 = _ext_euc_alg(b % a, a) 
     
    # Update x and y using results of recursive call 
    x = y1 - (b // a) * x1 
    y = x1 
     
    return gcd,x,y

def _inv_mult(a, m):
    '''
    Find modular inverses using EEA

            Parameters:
                    a : Inverse to be found
                    m : Modulus – must have (a, m) == 1

            Returns:
                    inv : Modular inverse of a mod m
    '''
    gcd, x, _ = _ext_euc_alg(a, m)
    
    if gcd == 1:
        return (x + m) % m
    

def _mod_pow(a, e, m):
    '''
    Compute modular exponentation under modular arithmetic
    
            Parameters:
                    a : Base
                    e : Exponent
                    m : Modulus
            
            Returns:
                    b : Value of a^e % m
    '''
    a, b = a % m, 1
    
    if a == 0: return 0
    
    while e > 0:
        if (e & 1) == 1:
            b = (a * b) % m
        
        e >>= 1
        a = (a * a) % m
    
    return b


def RSA_setup(p,q):
    '''
    Sets up parameters for RSA algorithm

            Parameters:
                    p : Large prime
                    q : Another large prime

            Returns:
                    (n, e, d) : RSA modulus, public key, and private key
    '''
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    # select public key s.t. (e, phi_n) == 1
    found = False
    
    while not found:
        e = random.randint(1, phi_n)
        if math.gcd(e, phi_n) == 1:
            found = True
    
    d = _inv_mult(e, phi_n)
    
    return n, e, d


def RSA_encrypt(m, e, n):
    '''
    Encrypts messages with RSA algorithm

            Parameters:
                    m : Integer representation of message – must be between 1 and n
                    e : Public key – selected as per RSA_setup
                    n : Modulus – selected as per RSA_setup

            Returns:
                    M : Encrypted message
    '''
    return _mod_pow(m, e, n)


def RSA_decrypt(M, d, n):
    '''
    Decrypts messages with RSA algorithm

            Parameters:
                    M : Encrypted message – calculated as per RSA_encrypt
                    d : Private key – selected as per RSA_setup
                    n : Modulus – selected as per RSA_setup

            Returns:
                    m : Plaintext message
    '''
    return _mod_pow(M, d, n)

In [57]:
n, e, d = RSA_setup(17,19) # n = 323
print(f"Public Key: {e}\nPrivate Key: {d}\n")

m = 104 # Remeber, must be between 1 - n

M = RSA_encrypt(m, e, n)
print(f"Encrypted Message: {M}")

m1 = RSA_decrypt(M, d, n)
print(f"Decrypted Message: {m}")

Public Key: 143
Private Key: 143

Encrypted Message: 264
Decrypted Message: 104
